In [1]:
from elasticsearch import Elasticsearch, helpers

client = Elasticsearch(
    "https://cea7dc701ec94051b4c6e7fcc316e6ec.us-central1.gcp.cloud.es.io",
    api_key="djRVQ1Fad0JUSjRHcjRwYXhHX0k6bWRycmw4b2dDTU9WMDRldkxxdXJRdw==",
)

In [6]:
# -----------------------------
# Define the index name and create index (if it doesn't exist)
# -----------------------------
index_name = "nation"
if not client.indices.exists(index=index_name):
    create_response = client.indices.create(index=index_name)
    print("Index created:", create_response)
else:
    print(f"Index '{index_name}' already exists.")




Index created: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'nation'}


In [5]:
client.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

In [ ]:
# -----------------------------
# Add or update mappings for the index
# -----------------------------
mappings = {
    "properties": {
        "text": {
            "type": "semantic_text"
        }
    }
}
mapping_response = client.indices.put_mapping(
    index=index_name,
    body=mappings
)
print("Mappings updated:", mapping_response)

In [12]:
# -----------------------------
# Sample documents to ingest
# -----------------------------
docs = [
    {
        "text": "Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acres across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site."
    },
    {
        "text": "Yosemite National Park is a United States National Park, covering over 750,000 acres of land in California. A UNESCO World Heritage Site, the park is best known for its granite cliffs, waterfalls and giant sequoia trees. Yosemite hosts over four million visitors in most years, with a peak of five million visitors in 2016. The park is home to a diverse range of wildlife, including mule deer, black bears, and the endangered Sierra Nevada bighorn sheep. The park has 1,200 square miles of wilderness, and is a popular destination for rock climbers, with over 3,000 feet of vertical granite to climb. Its most famous and cliff is the El Capitan, a 3,000 feet monolith along its tallest face."
    },
    {
        "text": "Rocky Mountain National Park is one of the most popular national parks in the United States. It receives over 4.5 million visitors annually, and is known for its mountainous terrain, including Longs Peak, which is the highest peak in the park. The park is home to a variety of wildlife, including elk, mule deer, moose, and bighorn sheep. The park is also home to a variety of ecosystems, including montane, subalpine, and alpine tundra. The park is a popular destination for hiking, camping, and wildlife viewing, and is a UNESCO World Heritage Site."
    }
]


In [13]:
for i, doc in enumerate(docs, start=1):
    result = client.index(
        index=index_name,
        document=doc,
        id=i,
        refresh=True
    )

    print( result)


{'_index': 'nation', '_id': '1', '_version': 2, 'result': 'updated', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'nation', '_id': '2', '_version': 2, 'result': 'updated', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}
{'_index': 'nation', '_id': '3', '_version': 2, 'result': 'updated', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 5, '_primary_term': 1}


In [15]:
# -----------------------------
# Bulk ingest documents
# -----------------------------
ingestion_timeout=300 # Allow time for semantic ML model to load
success, errors  = helpers.bulk(
    client.options(request_timeout=ingestion_timeout),
    docs,
    index=index_name,
    refresh="wait_for" # Wait until indexed documents are visible for search before returning the response
)
print(f"✅ Bulk indexed {success} documents into {index_name}")

if errors:
    print(f"⚠️  {len(errors)} errors occurred during bulk indexing")

✅ Bulk indexed 3 documents into nation


In [ ]:
# -----------------------------
# Define semantic search query
# -----------------------------
retriever_object = {
    "standard": {
        "query": {
            "semantic": {
                "field": "text",
                "query": "Sierra Nevada"
            }
        }
    }
}
search_response = client.search(
    index=index_name,
    retriever=retriever_object,
)
print(search_response['hits']['hits'])

[{'_index': 'national', '_id': '6AIgQZwBSx41yfn_OgL2', '_score': 9.605496, '_source': {'text': 'Yosemite National Park is a United States National Park, covering over 750,000 acres of land in California. A UNESCO World Heritage Site, the park is best known for its granite cliffs, waterfalls and giant sequoia trees. Yosemite hosts over four million visitors in most years, with a peak of five million visitors in 2016. The park is home to a diverse range of wildlife, including mule deer, black bears, and the endangered Sierra Nevada bighorn sheep. The park has 1,200 square miles of wilderness, and is a popular destination for rock climbers, with over 3,000 feet of vertical granite to climb. Its most famous and cliff is the El Capitan, a 3,000 feet monolith along its tallest face.'}}, {'_index': 'national', '_id': '6wIwQZwBSx41yfn_PAK_', '_score': 9.605496, '_source': {'text': 'Yosemite National Park is a United States National Park, covering over 750,000 acres of land in California. A UNE

In [20]:
client.get(index=index_name, id=1)

ObjectApiResponse({'_index': 'nation', '_id': '1', '_version': 2, '_seq_no': 3, '_primary_term': 1, 'found': True, '_source': {'text': 'Yellowstone National Park is one of the largest national parks in the United States. It ranges from the Wyoming to Montana and Idaho, and contains an area of 2,219,791 acres across three different states. Its most famous for hosting the geyser Old Faithful and is centered on the Yellowstone Caldera, the largest super volcano on the American continent. Yellowstone is host to hundreds of species of animal, many of which are endangered or threatened. Most notably, it contains free-ranging herds of bison and elk, alongside bears, cougars and wolves. The national park receives over 4.5 million visitors annually and is a UNESCO World Heritage Site.'}})